In [5]:
import os
import numpy as np
from skimage.feature import hog
from skimage import io
from skimage.transform import resize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Step 1: Data Preparation
train_dir = 'train/'
test_dir = 'test/'
validation_dir = 'validation/'

# Step 2: Feature Extraction
def extract_features_from_folder(folder, image_size=(128, 128)):
    features = []
    labels = []
    for class_name in os.listdir(folder):
        class_dir = os.path.join(folder, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image = io.imread(image_path, as_gray=True)
            resized_image = resize(image, image_size)  # Resize image to a fixed size
            hog_features = hog(resized_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
            features.append(hog_features)
            labels.append(class_name)
    return np.array(features), np.array(labels)

X_train, y_train = extract_features_from_folder(train_dir)
X_test, y_test = extract_features_from_folder(test_dir)
X_val, y_val = extract_features_from_folder(validation_dir)

# Step 3: Training
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 4: Validation
y_val_pred = clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Step 5: Testing
y_test_pred = clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)


Validation Accuracy: 0.6666666666666666
Test Accuracy: 0.4166666666666667


In [7]:
import pickle
with open('HoG with RF_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [10]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import pickle
from skimage.feature import hog
from skimage import io, transform

# Load the pre-trained Random Forest classifier
with open('HoG with RF_classifier.pkl', 'rb') as f:
    clf = pickle.load(f)

# Function to extract HoG features from an image
def extract_hog_features(image):
    resized_image = transform.resize(image, (128, 128))  # Resize image to a fixed size
    hog_features = hog(resized_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    return hog_features

# Function to classify image
def classify_image(image_path):
    try:
        image = io.imread(image_path, as_gray=True)
        hog_features = extract_hog_features(image)
        predicted_class = clf.predict([hog_features])[0]
        return predicted_class
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")
        return None

# Function to open file dialog and classify selected image
def classify_selected_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        try:
            predicted_class = classify_image(file_path)
            if predicted_class is not None:
                messagebox.showinfo("Classification Result", f"Predicted class: {predicted_class}")
                # Display the selected image
                img = Image.open(file_path)
                img.thumbnail((300, 300))
                img = ImageTk.PhotoImage(img)
                panel.config(image=img)
                panel.image = img
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

# Create GUI
root = tk.Tk()
root.title("Image Classification")
root.geometry("400x400")

# Button to open file dialog
classify_button = tk.Button(root, text="Select Image", command=classify_selected_image)
classify_button.pack(pady=10)

# Panel to display the selected image
panel = tk.Label(root)
panel.pack(fill="both", expand=True)

root.mainloop()
